In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


from hypergbm.estimators import XGBoostEstimator
from hypernets.pipeline.base import Pipeline
from hypernets.pipeline.transformers import FeatureGenerationTransformer
from hypernets.core.ops import ModuleChoice, HyperInput
from hypernets.core.search_space import HyperSpace
from hypernets.tabular.column_selector import column_exclude_datetime


def search_space(task=None):
    space = HyperSpace()
    with space.as_default():
        input = HyperInput(name='input1')
        feature_gen = FeatureGenerationTransformer(task=task,
                                                   trans_primitives=["add_numeric", "subtract_numeric", "divide_numeric", "multiply_numeric"])  # Add feature generation to search space
        full_pipeline = Pipeline([feature_gen], name=f'feature_gen_and_preprocess', columns=column_exclude_datetime)(input)
        xgb_est = XGBoostEstimator(fit_kwargs={})
        ModuleChoice([xgb_est], name='estimator_options')(full_pipeline)
        space.set_inputs(input)
    return space

In [2]:
from hypergbm import HyperGBM
from hypernets.searchers.evolution_searcher import EvolutionSearcher

rs = EvolutionSearcher(search_space,  200, 100, optimize_direction='max')
hk = HyperGBM(rs, task='multiclass', reward_metric='accuracy', callbacks=[])
hk.search(X_train, y_train, X_eval=X_eval, y_eval=y_eval)

In [3]:
estimator = hk.load_estimator(hk.get_best_trial().model_file)
y_pred = estimator.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

1.0